In [0]:
-- Query to get the count of items by specialization
DROP TABLE IF EXISTS dev.bi.s12n_item_counts_mgtv;
CREATE TABLE dev.bi.s12n_item_counts_mgtv AS (
-- Get the base s12n data
WITH base_s12n_data AS
(
SELECT 
    phoenix_specialization_id,
    base_phoenix_specialization_id,
    phoenix_specialization_version,
    phoenix_specialization_slug,
    phoenix_specialization_name,
    sequence_branch_id,
    is_specialization,
    is_professional_certificate,
    phoenix_specialization_primary_domain,
    phoenix_specialization_primary_subdomain,
    phoenix_specialization_primary_domain_id,
    phoenix_specialization_primary_subdomain_id,
    is_gateway_certificate,
    is_phoenix_specialization_currently_launched,
    phoenix_specialization_launch_ts,
    phoenix_specialization_display_page_launch_ts
 FROM
prod.gold.phoenix_specializations_vw
WHERE 
phoenix_specialization_id = base_phoenix_specialization_id
GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16
),

-- only get the count of items from the default branch
course_branch_item_counts AS
(
SELECT 
    cbi.course_id, 
    cbi.course_branch_id,  
    cbi.course_branch_item_type,
    COUNT(DISTINCT cbi.course_item_id) as number_of_items
FROM 
    prod.gold.course_branch_items_vw cbi
JOIN
    prod.gold.course_branches_vw cb
ON
    cbi.course_branch_id = cb.course_branch_id
AND
    cbi.course_id = cb.course_id
-- WHERE cb.is_course_branch_default 
where left(cbi.course_branch_id,7) not in ('branch~')
GROUP BY 1, 2, 3
)

-- Join this table to the base s12n data, after mapping the specialization to course IDs
SELECT 
    bsd.phoenix_specialization_id,
    bsd.base_phoenix_specialization_id,
    bsd.phoenix_specialization_version,
    bsd.phoenix_specialization_slug,
    bsd.phoenix_specialization_name,
    bsd.sequence_branch_id,
    bsd.is_specialization,
    bsd.is_professional_certificate,
    bsd.phoenix_specialization_primary_domain,
    bsd.phoenix_specialization_primary_subdomain,
    bsd.phoenix_specialization_primary_domain_id,
    bsd.phoenix_specialization_primary_subdomain_id,
    bsd.is_gateway_certificate,
    bsd.is_phoenix_specialization_currently_launched,
    bsd.phoenix_specialization_launch_ts,
    bsd.phoenix_specialization_display_page_launch_ts,
    sbc.course_id,
    sbc.sequence_branch_course_order,
    cbc.course_branch_item_type,
    cbc.number_of_items
FROM
    base_s12n_data bsd
JOIN
    prod.gold.sequence_branches_courses_vw sbc 
ON 
    bsd.sequence_branch_id = sbc.sequence_branch_id
JOIN
    course_branch_item_counts cbc
ON
    sbc.course_id = cbc.course_id
)

num_affected_rows,num_inserted_rows


In [0]:
DROP TABLE IF EXISTS dev.bi.s12n_item_info_mgtv;
CREATE TABLE dev.bi.s12n_item_info_mgtv AS 
WITH s12n_items AS (
  SELECT 
    base_phoenix_specialization_id,
    phoenix_specialization_slug,
    phoenix_specialization_primary_domain_id,
    phoenix_specialization_primary_subdomain_id,
    is_specialization,
    is_professional_certificate,
    is_gateway_certificate,
    DATE_TRUNC('month', phoenix_specialization_display_page_launch_ts)::DATE AS launch_month,
    course_branch_item_type AS item_type,
    SUM(number_of_items) AS number_of_items
  FROM 
  dev.bi.s12n_item_counts_mgtv
  WHERE phoenix_specialization_display_page_launch_ts IS NOT NULL
  GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9
),

s12n_courses AS
(
  SELECT 
       base_phoenix_specialization_id,
       COUNT(DISTINCT course_id) AS number_of_courses,
       SUM(number_of_items) AS total_items
    FROM
       dev.bi.s12n_item_counts_mgtv
    WHERE phoenix_specialization_display_page_launch_ts IS NOT NULL
    GROUP BY 1
),
add_course_item_count AS 
(
  SELECT 
    si.base_phoenix_specialization_id,
    si.phoenix_specialization_slug,
    si.phoenix_specialization_primary_domain_id,
    si.phoenix_specialization_primary_subdomain_id,
    si.is_specialization,
    si.is_professional_certificate,
    si.is_gateway_certificate,
    si.launch_month,
    si.item_type,
    si.number_of_items,
    sc.number_of_courses,
    sc.total_items
  FROM
      s12n_items si 
  JOIN
      s12n_courses sc
  ON
      si.base_phoenix_specialization_id = sc.base_phoenix_specialization_id
) 
select * from add_course_item_count

num_affected_rows,num_inserted_rows


# Item Pivot

In [0]:
create or replace table dev.bi.s12n_item_info_pivot_mgtv as 
WITH base_data AS (
  SELECT * FROM dev.bi.s12n_item_info_mgtv
)
SELECT *
FROM base_data
PIVOT (
  SUM(coalesce(number_of_items,0))
  FOR item_type IN (
    'discussionPrompt',
    'lecture',
    'ltiTool',
    'peerReview',
    'placeholder',
    'programmingAssignment',
    'quiz',
    'reading',
    'staffGraded',
    'ungradedLab',
    'widget'
  )
);

num_affected_rows,num_inserted_rows


# QA

In [0]:
select * from dev.bi.s12n_item_info_pivot_mgtv
where base_phoenix_specialization_id= 'KPSzVL9AEeyfhAreTccY9Q'

base_phoenix_specialization_id,phoenix_specialization_slug,phoenix_specialization_primary_domain_id,phoenix_specialization_primary_subdomain_id,is_specialization,is_professional_certificate,is_gateway_certificate,launch_month,number_of_courses,total_items,discussionPrompt,lecture,ltiTool,peerReview,placeholder,programmingAssignment,quiz,reading,staffGraded,ungradedLab,widget
KPSzVL9AEeyfhAreTccY9Q,meta-front-end-developer,computer-science,mobile-and-web-development,false,true,true,2022-05-01,9,991,33,344,null,5,null,8,3,373,190,35,null


In [0]:
-- select count(distinct base_phoenix_specialization_id) from dev.bi.s12n_item_info_pivot_mgtv where year(launch_month) >= 2020 and year(launch_month) <= 2024
-- where base_phoenix_specialization_id= 'KPSzVL9AEeyfhAreTccY9Q'
